In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import re
from nameparser import HumanName

In [2]:
def scrape_projections():
    players_soup = BeautifulSoup(urlopen('https://www.fantasypros.com/nfl/cheatsheets/half-ppr.php').read(), 'lxml')
    players_list = []
    position_dict = {'Quarterbacks': 'QB',
                     'Running Backs': 'RB',
                     'Wide Receivers': 'WR',
                     'Tight Ends': 'TE',
                     'Kickers': 'K'
                    }
    for pos_group in players_soup.find('div', {'class': 'position player-list'}).findAll('div', {'class': re.compile('three columns')}):
            num_players = len(pos_group.findAll('li'))
            print('Starting {0}\n'.format(pos_group.h5.string))
            for i, player in enumerate(pos_group.findAll('li')):
                if pos_group.h5.string in position_dict.keys():
                    p_proj = {"Name": player.a.string,
                              "Team": player.small.string,
                              "Position": position_dict[pos_group.h5.string]
                             }
                    link = 'https://www.fantasypros.com' + player.a['href'].replace('players','projections')
                    link = link + '?scoring=HALF' if p_proj['Position'] != 'QB' and p_proj['Position'] != 'K' else link
                    p_soup = BeautifulSoup(urlopen(link).read(), 'lxml')
                    p_table = p_soup.findAll('table', {'class': 'table table-bordered all-stats'})[1]
                    p_proj.update(dict(zip([cat.string for cat in p_table.find('thead').findAll('th')], [float(stat.string) for stat in p_table.find('tbody').findAll('td')])))
                    players_list.append(p_proj)
                    if i%10==0:
                        print('{0} of {1} completed.'.format(i, num_players))
    players_df = pd.DataFrame(players_list)
    
    defense_soup = BeautifulSoup(urlopen('https://www.fantasypros.com/nfl/projections/dst.php?week=draft').read(), 'lxml')
    defense_table = defense_soup.findAll('div', {'class':'mobile-table'})[0]
    categories_list = [cat.string for cat in defense_table.find('thead').findAll('th')][1:]
    categories_list[-1] = 'Points'
    defense_list = []
    for dst in defense_table.find('tbody').findAll('tr'):
        dst_dict = {"Name": dst.a.string.split(' ')[-1].strip(), 
                    "Position": 'DST'
                   }
        dst_dict.update(zip(categories_list, [float(stat.string.replace(',','')) if stat.string is not None else 0 for stat in dst.findAll('td', {'class':'center'})]))
        defense_list.append(dst_dict)
    players_df = players_df.append(defense_list)
    flex_positions = ['WR','RB','TE']
    players_df['Flex'] = players_df.Position.apply(lambda pos: True if pos in flex_positions else False)
    return players_df

In [3]:
players_df = scrape_projections()

Starting Quarterbacks

0 of 40 completed.
10 of 40 completed.


URLError: <urlopen error EOF occurred in violation of protocol (_ssl.c:749)>

In [ ]:
players_df.to_csv('fpros_player_projections.csv', index=False)
# players_df = pd.read_csv('fpros_player_projections.csv')
# players_df = players_df[['Name','Position','Points']].sort_values(by='Points',ascending=False).reset_index(drop=True)

In [ ]:
# number fire ROS projections link: https://www.numberfire.com/nfl/fantasy/remaining-projections
nf_players_df = pd.read_csv('nf_player_projections.csv')

In [ ]:
def calc_ppr_points(row):
    return row['StdPts'] + row['Rec']/2.0
def calc_ppr_ratio(row):
    return row['NF Points']/row['StdPts']
def calc_ppr_CI(row,CI):
    return row[CI]*row['Ratio']

nf_players_df['NF Points'] = nf_players_df.apply(lambda row: calc_ppr_points(row), axis=1)
nf_players_df['Ratio'] = nf_players_df.apply(lambda row: calc_ppr_ratio(row), axis=1)
nf_players_df['NF Lower CI'] = nf_players_df.apply(lambda row: calc_ppr_CI(row,'LowerCI'), axis=1)
nf_players_df['NF Upper CI'] = nf_players_df.apply(lambda row: calc_ppr_CI(row,'UpperCI'), axis=1)
nf_players_df = nf_players_df[['Name','Position','Team','NF Points','NF Lower CI','NF Upper CI']]
nf_players_df = nf_players_df.set_index(['Name','Position','Team'])

In [ ]:
full_name = players_df['Name']
players_df['Name'] = players_df['Name'].apply(lambda name: name[0]+'. ' + HumanName(name).last)
j_players_df = players_df.join(nf_players_df, on=['Name','Position','Team'])
j_players_df['FP Points'] = j_players_df['Points']
players_df = j_players_df[['Name','Position','Team', 'FP Points', 'NF Points', 'NF Lower CI', 'NF Upper CI']]

In [ ]:
players_df.head()

In [ ]:
players_df.head()

In [ ]:
j_players_df.head()

In [ ]:
def project_player(name, site, stat_type):
    if np.count_nonzero(players_df.Name.isin([name])):
        return players_df[players_df['Name'] == name]['{0} {1}'.format(site, stat_type)].iloc[0]
    else:
        return 0

In [ ]:
# league must be publicly viewable
def scrape_teams(league_link='http://games.espn.com/ffl/clubhouse?leagueId=1744757&teamId={0}&seasonId=2017'):
    position_list = ['QB','RB1','RB2','WR1','WR2','TE','FLEX','DST','K','Bench1','Bench2','Bench3','Bench4','Bench5','Bench6','Bench7']
    teams_list = []
    for i in range(1,11):
        try:
            req = Request(league_link.format(i), headers={ 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'})
            team_soup = BeautifulSoup(urlopen(req).read(), 'lxml')
            roster = {'Owner': team_soup.find('li',{'class':'per-info'}).string}
            roster.update(dict(zip(position_list, [player.a.string.replace(' D/ST','').replace(' Jr.', '').replace(' Sr.','') for player in team_soup.findAll('td', {'class': 'playertablePlayerName'})])))
            teams_list.append(roster)
            print("Scraped {0} Roster".format(roster['Owner']))
        except Exception as e:
            print(e.code)

    teams_df = pd.DataFrame(teams_list)
    for col in list(teams_df.columns[0:10]) + list(teams_df.columns[11:]):
            teams_df[col + '_FP_Projection'] = teams_df[col].apply(lambda p: project_player(p, 'FP', 'Points'))
            teams_df[col + '_NF_Projection'] = teams_df[col].apply(lambda p: project_player(p, 'NF', 'Points'))
            teams_df[col + '_NF_Lower_CI'] = teams_df[col].apply(lambda p: project_player(p, 'NF', 'Lower CI'))
            teams_df[col + '_NF_Upper_CI'] = teams_df[col].apply(lambda p: project_player(p, 'NF', 'Upper CI'))
    return teams_df

In [ ]:
teams_df = scrape_teams()

In [ ]:
def valid_substitute(name, position):
    if np.count_nonzero(players_df.Name.isin([name])):
        if players_df[players_df['Name'] == name]['Position'].iloc[0] == re.sub(r'\d+', '', position):
            return True
        if players_df[players_df['Name'] == name]['Flex'].iloc[0] and position == 'FLEX':
            return True
    if 'Bench' in position:
        return True
    else:
        return False

In [ ]:
def construct_new_team(team_dict, players_lost, players_acquired):
    new_team_dict = dict(team_dict)
    unfilled_positions = []
    
    for key, value in team_dict.items():
        if value in players_lost:
            unfilled_positions.append(key)
            new_team_dict[key] = ''
            new_team_dict[key + '_FP_Projection'] = 0
            new_team_dict[key + '_NF_Projection'] = 0
            new_team_dict[key + '_NF_Lower_CI'] = 0
            new_team_dict[key + '_NF_Upper_CI'] = 0
    
    players_acquired_iter = players_acquired
    moved = []
    unfilled_positions = [pos for pos in unfilled_positions if 'Bench' not in pos] + [pos for pos in unfilled_positions if 'Bench' in pos]
    
    while len(unfilled_positions) > 0:
        # first fill out starters with a combination of the acquired players and your bench, whichever is best
        u_pos = unfilled_positions[0]
        potential_subs = []
        
        for key, value in new_team_dict.items():
            if '_' not in key and value != '':
                if valid_substitute(value, u_pos) and value not in moved:
                    potential_subs.append(value)
        for p_acq in players_acquired_iter:
            if valid_substitute(p_acq, u_pos):
                potential_subs.append(p_acq)
        
        best_performer = '', 0
        for i, pot_sub in enumerate(potential_subs):
            pot_sub_proj = project_player(pot_sub, sort_by, 'Points')
            if pot_sub_proj > best_performer[1]:
                best_performer = pot_sub, pot_sub_proj

        if best_performer[0] in players_acquired_iter:
            players_acquired_iter.remove(best_performer[0])
            unfilled_positions.pop(0)
            moved.append(best_performer[0])
            
        else:
            for key in new_team_dict.keys():
                if best_performer[0] == new_team_dict[key]:
                    new_team_dict[key] = ''
                    new_team_dict[key + '_FP_Projection'] = 0
                    new_team_dict[key + '_NF_Projection'] = 0
                    new_team_dict[key + '_NF_Lower_CI'] = 0
                    new_team_dict[key + '_NF_Upper_CI'] = 0
                    unfilled_positions.pop(0)
                    unfilled_positions.append(key)
                    moved.append(best_performer[0])
                    
        new_team_dict[u_pos] = str(best_performer[0])
        new_team_dict[u_pos + '_FP_Projection'] = project_player(new_team_dict[u_pos],'FP','Points')
        new_team_dict[u_pos + '_NF_Projection'] = project_player(new_team_dict[u_pos],'NF','Points')
        new_team_dict[u_pos + '_NF_Lower_CI'] = project_player(new_team_dict[u_pos],'NF','Lower CI')
        new_team_dict[u_pos + '_NF_Upper_CI'] = project_player(new_team_dict[u_pos],'NF','Upper CI')
    
    
        if all([True if 'Bench' in val else False for val in unfilled_positions]):
            # only bench players left to fill up
            break
    
    while len(unfilled_positions) > 0:
        # then fill out bench with the displaced players
        # this loop handles the event where players  are traded to your team that weren't good enough to start
        # this case is handled by taking these players and filling out the remainder of your bench
        u_pos = unfilled_positions[0]
        new_team_dict[u_pos] = str(players_acquired_iter[0])
        new_team_dict[u_pos + '_FP_Projection'] = project_player(new_team_dict[u_pos],'FP','Points')
        new_team_dict[u_pos + '_NF_Projection'] = project_player(new_team_dict[u_pos],'NF','Points')
        new_team_dict[u_pos + '_NF_Lower_CI'] = project_player(new_team_dict[u_pos],'NF','Lower CI')
        new_team_dict[u_pos + '_NF_Upper_CI'] = project_player(new_team_dict[u_pos],'NF','Upper CI')
        unfilled_positions.pop(0)
        players_acquired_iter.pop(0)
    
    return new_team_dict

In [ ]:
def project_team(team_dict, sort_by='NF'):
    bench_proj = 0
    starters_proj = 0
    for key in team_dict.keys():
        if '{0}_Projection'.format(sort_by) in key and 'Bench' in key:
            bench_proj += team_dict[key]
        if '{0}_Projection'.format(sort_by) in key and 'Bench' not in key:
            starters_proj += team_dict[key]
    return starters_proj, bench_proj

In [ ]:
def analyze_trade(owner_1, owner_2, team_1_players, team_2_players, sort_by='NF'):
    team_1_dict = teams_df[teams_df['Owner'] == owner_1].to_dict('records')[0]
    team_2_dict = teams_df[teams_df['Owner'] == owner_2].to_dict('records')[0]
    team_1_prev_proj = project_team(team_1_dict)
    team_2_prev_proj = project_team(team_2_dict)
    team_1_new_dict = construct_new_team(team_1_dict, team_1_players.copy(), team_2_players.copy())
    team_2_new_dict = construct_new_team(team_2_dict, team_2_players.copy(), team_1_players.copy())
    team_1_new_proj = project_team(team_1_new_dict)
    team_2_new_proj = project_team(team_2_new_dict)   
    
    team_1_diff = tuple(np.subtract(team_1_new_proj, team_1_prev_proj))
    team_2_diff = tuple(np.subtract(team_2_new_proj, team_2_prev_proj))
    
    positions = ['QB','RB1','RB2','WR1','WR2','TE','FLEX','K','DST','Bench1','Bench2','Bench3','Bench4','Bench5','Bench6']
    
    print('Before the Trade:')
    print('Team 1 - Starters: {:05.2f} Bench: {:05.2f}'.format(*team_1_prev_proj))
    print('Team 2 - Starters: {:05.2f} Bench: {:05.2f}\n'.format(*team_2_prev_proj))
    print('After the Trade:')
    print('Team 1 - Starters: {:05.2f} Bench: {:05.2f}'.format(*team_1_new_proj))
    print('Team 2 - Starters: {:05.2f} Bench: {:05.2f}\n'.format(*team_2_new_proj))
    print('Net Differences:')
    print('Team 1 - Starters: {:05.2f} Bench: {:05.2f}'.format(*team_1_diff))
    print('Team 2 - Starters: {:05.2f} Bench: {:05.2f}\n'.format(*team_2_diff))
    print('Team 1 - Roster:')
    print('{:>6}: '.format('Position') + '{:20}'.format('Player Name') + '{0} Projection'.format(sort_by) + 'NF Confidence Interval')
    for pos in positions:
        output_str = '{:>8}: '.format(pos)
        output_str += '{:20}'.format(team_1_new_dict[pos])
        output_str += '{:05.2f}'.format(team_1_new_dict[pos+'_Projection'], team_1_new_dict)
        output_str += '   ({:05.2f}~{:05.2f})'.format(team_1_new_dict[pos+'_NF_Lower_CI'],team_1_new_dict[pos+'_NF_Upper_CI'])
        print(output_str)
    print('\n')
    print('Team 2 - Roster:')
    for pos in positions:
        output_str = '{:>8}: '.format(pos)
        output_str += '{:20}'.format(team_1_new_dict[pos])
        output_str += '{:05.2f}'.format(team_1_new_dict[pos+'_Projection'], team_1_new_dict)
        output_str += '   ({:05.2f}~{:05.2f})'.format(team_1_new_dict[pos+'_NF_Lower_CI'],team_1_new_dict[pos+'_NF_Upper_CI'])
        print(output_str)

In [ ]:
analyze_trade('Surya Bahubalendruni', 'Aaron Zephir', ['Delanie Walker','Tyreek Hill', 'Tom Brady'], ['Travis Kelce', 'John Brown', 'Aaron Rodgers'])